# LSTM Classifier
***

In [159]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.random import set_seed
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
import talos
from talos.utils import lr_normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import metrics
import keras.backend as K
from functools import partial


Checking whether tensorflow is running on GPU:

In [110]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.test.gpu_device_name())

Num GPUs Available:  1
/device:GPU:0


Importing final dataset:

In [111]:
df = pd.read_csv('datasets/final_db.csv', index_col=0)
df.head(3)

,target,mean_sent,STOCH_slowk,STOCH_slowd,MACD,MACD_hist,CCI_5,CCI_10,CCI_21,CCI_50,MOM_5,MOM_10,MOM_21,RSI_5,RSI_10,RSI_21,RSI_50
Day,,,,,,,,,,,,,,,,,
2015-01-01,0.0,0.011820,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454
2015-01-02,1.0,0.009464,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454
2015-01-03,0.0,0.009646,67.198744,57.422805,-12.540186,4.426599,-76.487357,-58.930944,-57.963081,-8.626979,-28.059998,-48.588989,-80.842987,30.94113,30.495281,36.523877,43.526454


---
### Model & hyperparameter tuning with Talos

In [166]:
# Function to get class weights:
def cwts(data):
    c0, c1 = np.bincount(data['target'])
    #making the weights inversely proportional to the amount of observations:
    w0, w1 = (1/c0)*(len(data))/2, (1/c1)*(len(data))/2
    return {0: w0, 1:w1}

# Defining Recall, Precision, Specificity and F1 metrics:
#(https://medium.com/analytics-vidhya/custom-metrics-for-keras-tensorflow-ae7036654e05)
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras


def specificity(y_true, y_pred):
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    return tn / (tn + fp + K.epsilon())


def f1(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

#========================================================================================#
# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return np.clip(lr * tf.math.exp(-0.1), 0.000001, 0.001) #limited to these learning rates

#========================================================================================#
#LSTM ARCHITECTURE
def lstm_model(x_train, y_train, x_val, y_val, params):

    tf.keras.backend.clear_session()
    #This function defines the LSTM model
    win_length = params['window']
    batch_size = params['batch_size']
    num_features= x_train.shape[1]
    train_generator = TimeseriesGenerator(x_train, y_train, length= win_length, sampling_rate = 1, batch_size= batch_size)
    test_generator = TimeseriesGenerator(x_val, y_val, length= win_length, sampling_rate = 1, batch_size= batch_size)
    #==================================================================#
    # MODEL ARCHITECTURE
    model = Sequential()
    
    #Input and first LSTM layer:
    model.add(LSTM(params['neurons'],input_shape = (win_length, num_features), return_sequences = True))
    
    # Dropout layer after LSTM layer
    model.add(Dropout(params['dropout']))

    # Another stacked LSTM layer, to allow for complexity
    model.add(LSTM(params['neurons'], return_sequences=True))
    # Dropout layer after LSTM layer
    model.add(Dropout(params['dropout']))

    
    # Third layer to receive all information
    model.add(LSTM(params['neurons'], return_sequences=False))

    # DEnse layer to yield all info to output, reducing dimension
    if params['activation'] == 'leakyrelu':
        model.add(Dense(params['dense_neurons'], activation = tf.keras.layers.LeakyReLU(alpha=0.01)))#output layer
    else:
        model.add(Dense(params['dense_neurons'], activation = params['activation']))#output layer

    
    # Output layer (class =0 or 1)
    model.add(Dense(1, activation = 'sigmoid'))#output layer
    
    #==================================================================#
    # Defining Early Stopping to avoid overfitting (after 3 attempts)
    early_stopping = EarlyStopping(monitor='val_loss',patience = 3, mode='min')

    # Using exponential decay:
    
    callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    # Compiling
    # (lr=lr_normalizer(params['lr'], params['optimizer']))
    # callbacks = [early_stopping, callbacklr],
    model.compile(loss='binary_crossentropy', 
                    optimizer = params['optimizer'], 
                    metrics=['accuracy',precision,recall,f1,specificity,])
    
    history = model.fit(train_generator,
                        validation_data=test_generator, # validation sample with features
                        epochs = params['epochs'], # validation sample with labels
                        shuffle=False, callbacks = [early_stopping, callbacklr],
                        class_weight = class_weight, # imbalanced data
                        verbose=2
                        )

    return(history, model)

To facilitate computation we scale the features.

In [167]:
# We scale the data so hasten the LSTM's conversion
scaler = MinMaxScaler() 

dffinal_scaled = scaler.fit_transform(df)

# Features x Target
features = dffinal_scaled
target = dffinal_scaled[:,0]

# Shuffle is set to False because it is a timeseries, and the order matters.
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0, shuffle = False)

We then define the class weights (imbalanced):

In [168]:
class_weight = cwts(df)
class_weight

{0: 1.1066666666666667, 1: 0.9120879120879121}

In [170]:
# Defining parameters considered in the hyperparameter tuning
params = {'lr': [0.01, 0.001],
        'neurons': [32, 64, 128], 
        'dropout': [0, 0.3, 0.5],
        'batch_size': [8, 32, 64, 128, 512], 
        'epochs': [25, 50],
        'optimizer': ['Nadam', 'Adam','SGD'],
        'activation': ['leakyrelu','relu','elu'],
        'dense_neurons': [8, 16, 32],
        'window': [5,10,21,50],
        }

# Scanning for best hyperparameters with Talos
scan = talos.Scan(x = x_train, y = y_train, x_val = x_test, y_val = y_test, params = params, model = lstm_model,experiment_name = 'lstm_model_val')

Epoch 1/25
182/182 - 9s - loss: 0.6925 - accuracy: 0.5230 - precision: 0.3807 - recall: 0.6735 - f1: 0.4763 - specificity: 0.3204 - val_loss: 0.6925 - val_accuracy: 0.5208 - val_precision: 0.5109 - val_recall: 0.9783 - val_f1: 0.6577 - val_specificity: 0.0000e+00 - lr: 0.0010 - 9s/epoch - 49ms/step
Epoch 2/25
182/182 - 3s - loss: 0.6923 - accuracy: 0.5010 - precision: 0.3056 - recall: 0.5158 - f1: 0.3709 - specificity: 0.4834 - val_loss: 0.6935 - val_accuracy: 0.4792 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00 - val_specificity: 1.0000 - lr: 0.0010 - 3s/epoch - 19ms/step
Epoch 3/25
182/182 - 3s - loss: 0.6914 - accuracy: 0.5168 - precision: 0.3431 - recall: 0.5402 - f1: 0.4014 - specificity: 0.4472 - val_loss: 0.6933 - val_accuracy: 0.4986 - val_precision: 0.0960 - val_recall: 0.0732 - val_f1: 0.0776 - val_specificity: 0.9377 - lr: 0.0010 - 3s/epoch - 19ms/step
Epoch 4/25
